In [2]:
from google.colab import drive
!mkdir /content/data
drive.mount("/content/data")

mkdir: cannot create directory ‘/content/data’: File exists
Drive already mounted at /content/data; to attempt to forcibly remount, call drive.mount("/content/data", force_remount=True).
mkdir: cannot create directory ‘/content/data’: File exists
Drive already mounted at /content/data; to attempt to forcibly remount, call drive.mount("/content/data", force_remount=True).


In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-02 08:29:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.86.186.182, 35.172.177.65, 34.204.22.7, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.86.186.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.4’

ngrok-stable-linux- 100%[===================>]  14.28M  4.94MB/s    in 2.9s    

2019-04-02 08:29:07 (4.94 MB/s) - ‘ngrok-stable-linux-amd64.zip.4’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [4]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization, MaxPooling2D,Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os
import numpy as np
np.random.seed(1337)  # for reproducibility
import pickle
from keras.models import Sequential
# from tensorflow.python.keras import Sequential
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
K.set_image_data_format('channels_first')
!rm -r Graph
!rm -r GraphODE

Using TensorFlow backend.


rm: cannot remove 'GraphODE': No such file or directory


In [5]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://f6b66f91.ngrok.io


In [7]:
from binary_ops import binary_tanh as binary_tanh_op


H = 1.
kernel_lr_multiplier = 'Glorot'

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# nn
# batch_size = 50
batch_size = 32
epochs = 50
nb_channel = 1
# img_rows = 71 
# img_cols = 64
img_rows = 64 
img_cols = 64
nb_filters = 32 
nb_conv = 3
nb_pool = 2
nb_hid = 128
nb_classes = 151
use_bias = False

# learning rate schedule
lr_start = 1e-3
lr_end = 1e-4
lr_decay = (lr_end / lr_start)**(1. / epochs)

# BN
epsilon = 1e-6
momentum = 0.9

# dropout
p1 = 0.25
p2 = 0.5
# data needs to be a tuple with two elements.
# the data, shuffled and split between train and test sets
# (X_train, y_train), (X_test, y_test) = load_faces95_data.load_data()
# pickledDataset = open("faces95dataWithPCA.pickle","rb")
pickledDataset = open("/content/data/My Drive/Minor Project/toSend/faces96dataWithImgSalMBD.pickle","rb")
data = pickle.load(pickledDataset)
(X_train, y_train), (X_test, y_test) = data['res']
print("Dataset has been loaded")

X_train = X_train.reshape(2567, 1, 64, 64)
X_test = X_test.reshape(453, 1, 64, 64)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes) * 2 - 1 # -1 or 1 for hinge loss
Y_test = np_utils.to_categorical(y_test, nb_classes) * 2 - 1


Dataset has been loaded
2567 train samples
453 test samples


In [8]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),   
                     padding='same', use_bias=use_bias, name='conv4'))
# remove 3 lines below if needed
# model.add(Conv2D(256, kernel_size=(3, 3),   
#                      padding='same', use_bias=use_bias, name='conv4_a'))
# model.add(Conv2D(256, kernel_size=(3, 3),   
#                      padding='same', use_bias=use_bias, name='conv4_b'))
# model.add(Conv2D(256, kernel_size=(3, 3),   
#                      padding='same', use_bias=use_bias, name='conv4_c'))
model.add(MaxPooling2D(pool_size=(2, 2), name='pool4'))
model.add(BatchNormalization(epsilon=epsilon, momentum=momentum, name='bn4'))
model.add(Activation('selu', name='act4'))
model.add(Flatten())
# dense1
model.add(Dense(1024,   use_bias=use_bias, name='dense5'))
model.add(BatchNormalization(epsilon=epsilon, momentum=momentum, name='bn5'))
model.add(Activation('selu', name='act5'))
# dense2
model.add(Dense(nb_classes,   use_bias=use_bias, name='dense6'))
model.add(BatchNormalization(epsilon=epsilon, momentum=momentum, name='bn6'))

opt = Adam(lr=lr_start) 
model.compile(loss='squared_hinge', optimizer=opt, metrics=['acc'])

lr_scheduler = LearningRateScheduler(lambda e: lr_start * lr_decay ** e)

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(X_test, Y_test),
                    callbacks=[lr_scheduler,TensorBoardColabCallback(tbc)])

model.save('/content/data/My Drive/Minor Project/toSend/saved_models/CNNF96_colab.h5')

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 2567 samples, validate on 453 samples
Epoch 1/50
2567/2567 [==============================] - 10s 4ms/step - loss: 1.5406 - acc: 0.3522 - val_loss: 0.5843 - val_acc: 0.4901
Epoch 2/50
2567/2567 [==============================] - 8s 3ms/step - loss: 1.1975 - acc: 0.7191 - val_loss: 0.9269 - val_acc: 0.8411
Epoch 3/50
2567/2567 [==============================] - 8s 3ms/step - loss: 1.0187 - acc: 0.8126 - val_loss: 1.0345 - val_acc: 0.9272
Epoch 4/50
2567/2567 [==============================] - 8s 3ms/step - loss: 0.8553 - acc: 0.8699 - val_loss: 0.8644 - val_acc: 0.9205
Epoch 5/50
2567/2567 [==============================] - 8s 3ms/step - loss: 0.7194 - acc: 0.9042 - val_loss: 0.4820 - val_acc: 0.9669
Epoch 6/50
2567/2567 [==============================] - 8s 3ms/step - loss: 0.6223 - acc: 0.9314 - val_loss: 0.5878 - val_acc: 0.9801
Epoch 7/50
2567/2567 [==============================] - 8s 3ms/step - loss: 0.5261 - acc: 0.9552 - v